# Lab 4-2: Load Data

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    We use elemental PyTorch to implement linear regression here. However, in most actual applications, abstractions such as <code>nn.Module</code> or <code>nn.Linear</code> are used.
</div>

## Slicing 1D Array

In [1]:
nums = [0, 1, 2, 3, 4]

In [2]:
print(nums)

[0, 1, 2, 3, 4]


index 2에서 4 전까지 가져와라. (앞 포함, 뒤 비포함)

In [3]:
print(nums[2:4])

[2, 3]


index 2부터 다 가져와라.

In [4]:
print(nums[2:])

[2, 3, 4]


index 2 전까지 가져와라. (역시 뒤는 비포함)

In [5]:
print(nums[:2])

[0, 1]


전부 가져와라

In [6]:
print(nums[:])

[0, 1, 2, 3, 4]


마지막 index 전까지 가져와라. (뒤는 비포함!)

In [7]:
print(nums[:-1])

[0, 1, 2, 3]


assign 도 가능!

In [8]:
nums[2:4] = [8, 9]

In [9]:
print(nums)

[0, 1, 8, 9, 4]


## Slicing 2D Array

In [10]:
import numpy as np

In [11]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [12]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [13]:
b[:, 1]

array([ 2,  6, 10])

In [14]:
b[-1]

array([ 9, 10, 11, 12])

In [15]:
b[-1, :]

array([ 9, 10, 11, 12])

In [16]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [17]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

## Loading Data from `.csv` file

In [18]:
import numpy as np

In [19]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [20]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [21]:
print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [22]:
print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


## Imports

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [24]:
# For reproducibility
torch.manual_seed(1)

## Low-level Implementation

In [25]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.529297
Epoch    2/20 Cost: 3675.299072
Epoch    3/20 Cost: 1366.261108
Epoch    4/20 Cost: 512.542236
Epoch    5/20 Cost: 196.896484
Epoch    6/20 Cost: 80.190910
Epoch    7/20 Cost: 37.038647
Epoch    8/20 Cost: 21.081354
Epoch    9/20 Cost: 15.178741
Epoch   10/20 Cost: 12.993667
Epoch   11/20 Cost: 12.183028
Epoch   12/20 Cost: 11.880545
Epoch   13/20 Cost: 11.765955
Epoch   14/20 Cost: 11.720857
Epoch   15/20 Cost: 11.701424
Epoch   16/20 Cost: 11.691505
Epoch   17/20 Cost: 11.685121
Epoch   18/20 Cost: 11.680006
Epoch   19/20 Cost: 11.675381
Epoch   20/20 Cost: 11.670943


## High-level Implementation with `nn.Module`

In [26]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [27]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 28693.488281
Epoch    1/20 Cost: 10618.750000
Epoch    2/20 Cost: 3936.015137
Epoch    3/20 Cost: 1465.219360
Epoch    4/20 Cost: 551.693909
Epoch    5/20 Cost: 213.934677
Epoch    6/20 Cost: 89.052261
Epoch    7/20 Cost: 42.876019
Epoch    8/20 Cost: 25.799616
Epoch    9/20 Cost: 19.482409
Epoch   10/20 Cost: 17.143106
Epoch   11/20 Cost: 16.274515
Epoch   12/20 Cost: 15.949698
Epoch   13/20 Cost: 15.825970
Epoch   14/20 Cost: 15.776586
Epoch   15/20 Cost: 15.754649
Epoch   16/20 Cost: 15.742906
Epoch   17/20 Cost: 15.734896
Epoch   18/20 Cost: 15.728317
Epoch   19/20 Cost: 15.722223
Epoch   20/20 Cost: 15.716329


## Dataset and DataLoader

너무 데이터가 크면 `x_data`, `y_data` 를 전부 다 가져오지 말고, 필요한 배치만 가져올 수 밖에 없다.

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [29]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

In [30]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

[PyTorch Data Loading and Processing tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#iterating-through-the-dataset)

In [31]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [32]:
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [33]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 46978.785156
Epoch    0/20 Batch 2/3 Cost: 6497.784668
Epoch    0/20 Batch 3/3 Cost: 5174.395996
Epoch    1/20 Batch 1/3 Cost: 711.106018
Epoch    1/20 Batch 2/3 Cost: 401.381378
Epoch    1/20 Batch 3/3 Cost: 212.294891
Epoch    2/20 Batch 1/3 Cost: 26.882061
Epoch    2/20 Batch 2/3 Cost: 0.057693
Epoch    2/20 Batch 3/3 Cost: 41.471272
Epoch    3/20 Batch 1/3 Cost: 13.444824
Epoch    3/20 Batch 2/3 Cost: 14.656877
Epoch    3/20 Batch 3/3 Cost: 5.495739
Epoch    4/20 Batch 1/3 Cost: 6.619390
Epoch    4/20 Batch 2/3 Cost: 19.459251
Epoch    4/20 Batch 3/3 Cost: 16.812302
Epoch    5/20 Batch 1/3 Cost: 19.544233
Epoch    5/20 Batch 2/3 Cost: 21.681820
Epoch    5/20 Batch 3/3 Cost: 13.138634
Epoch    6/20 Batch 1/3 Cost: 2.338879
Epoch    6/20 Batch 2/3 Cost: 21.505651
Epoch    6/20 Batch 3/3 Cost: 17.908745
Epoch    7/20 Batch 1/3 Cost: 8.261931
Epoch    7/20 Batch 2/3 Cost: 12.769864
Epoch    7/20 Batch 3/3 Cost: 24.187618
Epoch    8/20 Batch 1/3 Cost: 10.52